<a href="https://colab.research.google.com/github/lleiva25/Deep-Learning-Challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #Install modules fir clearing Jupyter GPU data
# !pip install numba
# !apt-get install nvidia-cuda-toolkit
# !export NUMBA_CUDA_DRIVER=/usr/local/cuda/lib64/libcudart.so

# #Restart the Jupyter Notebook
# !jupyter notebook stop
# !jupyter notebook

# #Import dependencies
# from numba import cuda

# #Clear GPU data
# device = cuda.get_current_device()
# device.reset()

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


<h1>Optimizing Method [71% to 77%] </h1>

In [ ]:
#Drop non-essential columns
#increased accuracy by 1%
app_df = application_df.drop(columns=['EIN',
                                      #'NAME',
                                     # 'AFFILIATION',#76%
                                      #'USE_CASE',
                                      #'ORGANIZATION',
                                      #'STATUS',
                                      #'INCOME_AMT',
                                      'SPECIAL_CONSIDERATIONS',
                                      'ASK_AMT'
                                      ])

app_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,1


In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_ntype = app_df['APPLICATION_TYPE'].value_counts()
app_ntype

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutapp_ntypeoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_ntype[app_ntype < 700].index)

# Replace in dataframe
for app in application_types_to_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

app_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_vcount =  app_df['CLASSIFICATION'].value_counts()
classification_vcount

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_vcount[classification_vcount < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
income_vcount =  app_df['INCOME_AMT'].value_counts()

# # Choose a cutoff value and create a list of classifications to be replaced
# # use the variable name `ask_amt_to_replace`
income_to_replace = list(income_vcount[income_vcount < 200].index)

# # Replace in dataframe
for income in income_to_replace:
     app_df['INCOME_AMT'] = app_df['INCOME_AMT'].replace(income,"Other")

# # Check to make sure binning was successful
app_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
Other              324
10M-50M            240
Name: INCOME_AMT, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
name_vcount =  app_df['NAME'].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `ask_amt_to_replace`
name_to_replace = list(name_vcount[name_vcount < 50].index)

# Replace in dataframe
for name in name_to_replace:
    app_df['NAME'] = app_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
app_df['NAME'].value_counts()

Other                                                                 24545
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(app_df,dtype=float)

app_dummies.head()

,STATUS,IS_SUCCESSFUL,NAME_AIR FORCE ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CALIFORNIA GARDEN CLUBS INC,NAME_CISD EDUCATIONAL SUPPORT GROUPS INC,NAME_CIVITAN INTERNATIONAL,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# Split our preprocessed data into our features and target arrays
X = app_dummies.drop(['IS_SUCCESSFUL'],axis=1).values
y = app_dummies['IS_SUCCESSFUL'].values

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=50
                                                    )

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 360
hidden_nodes_layer2 = 180
hidden_nodes_layer3 = 90
hidden_nodes_layer4 = 70
hidden_nodes_layer5 = 60
hidden_nodes_layer6 = 30
hidden_nodes_layer7 = 15
outer_nodes_layer = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=number_input_features, units=hidden_nodes_layer1, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "leaky_relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation = "sigmoid")) #76%

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation = "sigmoid")) #76%

# Fifth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation = "sigmoid"))

# Sixth hidden layer
##nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation = "sigmoid"))

# Seventh hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, activation = "sigmoid"))

#Add Dropout Layer to prevent overfitting
#None of the layers were overfitted since Parameters = 0
#nn.add(Dropout(0.25))

# Output layer #
nn.add(tf.keras.layers.Dense(units=outer_nodes_layer, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 360)               33120     
                                                                 
 dense_5 (Dense)             (None, 180)               64980     
                                                                 
 dense_6 (Dense)             (None, 90)                16290     
                                                                 
 dense_7 (Dense)             (None, 1)                 91        
                                                                 
Total params: 114481 (447.19 KB)
Trainable params: 114481 (447.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Compile the model
nn.compile(#loss = 'mean_squared_error', #73
           loss = 'binary_crossentropy', #73
           #loss='sparse_categorical_crossentropy', #47%
          # loss='categorical_crossentropy', #46%
           #loss = 'binary_crossentropy', #73%
           #optimizer = 'adam', #73
           #optimizer ='rmsprop', #73
           #optimizer='sgd', #46
           #optimizer='adagrad', #73
           #optimizer='adadelta', #73
           optimizer='adamax', #73
           #optimizer='nadam', #72
           metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,validation_split=0.15,batch_size=10)

Epoch 1/50
2187/2187 [==============================] - 8s 3ms/step - loss: 0.4869 - accuracy: 0.7580 - val_loss: 0.4676 - val_accuracy: 0.7746
Epoch 2/50
2187/2187 [==============================] - 7s 3ms/step - loss: 0.4830 - accuracy: 0.7576 - val_loss: 0.4644 - val_accuracy: 0.7792
Epoch 3/50
2187/2187 [==============================] - 7s 3ms/step - loss: 0.4811 - accuracy: 0.7600 - val_loss: 0.4748 - val_accuracy: 0.7805
Epoch 4/50
2187/2187 [==============================] - 7s 3ms/step - loss: 0.4806 - accuracy: 0.7592 - val_loss: 0.4614 - val_accuracy: 0.7810
Epoch 5/50
2187/2187 [==============================] - 7s 3ms/step - loss: 0.4785 - accuracy: 0.7614 - val_loss: 0.4596 - val_accuracy: 0.7813
Epoch 6/50
2187/2187 [==============================] - 7s 3ms/step - loss: 0.4781 - accuracy: 0.7631 - val_loss: 0.4599 - val_accuracy: 0.7782
Epoch 7/50
2187/2187 [==============================] - 7s 3ms/step - loss: 0.4770 - accuracy: 0.7632 - val_loss: 0.4605 - val_accuracy:

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.4730 - accuracy: 0.7736 - 435ms/epoch - 2ms/step
Loss: 0.4730331301689148, Accuracy: 0.7736443281173706


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimalized.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
